<a href="https://colab.research.google.com/github/bishram-acharya/Cross-Platform-Fingerprint-Matching-using-CNN/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time

# Preprocess function


def process_single_image(inpath):

    myKernel = np.ones((5, 5))

    img = cv2.imread(inpath)
    # reading the image in grayscale mode
    gray_img = cv2.imread(inpath, cv2.IMREAD_GRAYSCALE)
    opening_gray_img = cv2.morphologyEx(
        gray_img, cv2.MORPH_OPEN, myKernel)  # erosion followed by dilation
    opening_rgb_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, myKernel)

    mask = cv2.adaptiveThreshold(
        opening_gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 41, 4)

    mask = mask.astype(np.uint8)
    mask[mask == 255] = cv2.GC_FGD
    mask[mask == 0] = cv2.GC_BGD

    bgd_model = np.zeros((1, 65), dtype=np.float64)
    fgd_model = np.zeros((1, 65), dtype=np.float64)
    segmented_mask, _, _ = cv2.grabCut(
        opening_rgb_img, mask, None, bgd_model, fgd_model, iterCount=2, mode=cv2.GC_INIT_WITH_MASK)

    segmented_mask = np.where((segmented_mask == cv2.GC_FGD) | (
        segmented_mask == cv2.GC_PR_FGD), 255, 0).astype(np.uint8)

    segmented_image = cv2.bitwise_and(
        opening_rgb_img, opening_rgb_img, mask=segmented_mask)

    clahe = cv2.createCLAHE(clipLimit=30, tileGridSize=(7, 7))
    clahe_adjusted_image = clahe.apply(
        cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY))
    _, thresholded_img = cv2.threshold(
        clahe_adjusted_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    final_rotated_img = cv2.rotate(
        thresholded_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    downscaled_final_img = cv2.resize(
        final_rotated_img, (224, 224), interpolation=cv2.INTER_LANCZOS4)
    # this interpolation is computationally expensive, can try CUBIC for downscaling

    return downscaled_final_img


# Bulk processing
image_folder = sys.argv[1]
output_folder = sys.argv[2]
image_list = []
count = 1
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
start_time = time.time()
for filename in os.listdir(image_folder):

    inpath = f'D:/datset_fingerprint_buffalo/Fingerprint_Train_Test_Split/Task1/Train/Contactless/{filename}'
    downscaled = process_single_image(inpath)
    filename = f'D:/datset_fingerprint_buffalo/Fingerprint_Train_Test_Split/Task1/Train/segmented_contactless/{filename}'
    # Overwrite
    cv2.imwrite(filename, downscaled)
    print("Done")
end_time = time.time()
print(end_time-start_time)
